In [1]:
# Cell 1: Setup
%load_ext autoreload
%autoreload 2

import sys
import gc
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score

# Setup path
sys.path.append(str(Path.cwd().parent))
from src import config, preprocessing, autoencoder, rf_classifier, evaluation, utils

# Tạo folder lưu kết quả riêng cho Cross-Test
exp_paths = utils.setup_experiment_folder()
print(f"📂 Saving results to: {exp_paths['root']}")

ModuleNotFoundError: No module named 'src'

In [ ]:
# ==============================================================================
# KỊCH BẢN 1: TRAIN TRÊN 2017 -> TEST TRÊN 2018
# ==============================================================================

# Cell 2: Load Data 2017 (TRAIN)
print("\n--- 1. LOADING TRAIN SET (CIC-IDS2017) ---")
# Hàm này trả về DataFrame, ta giữ nguyên để dễ kiểm soát lúc đầu
df_train, y_train = preprocessing.load_single_dataset_year('2017', binary_mode=True)

# Chuyển sang numpy để đưa vào model
X_train = df_train.values

print(f"Training Data Shape: {X_train.shape}")
print(f"Label Distribution: {pd.Series(y_train).value_counts()}")

In [ ]:
# Cell 3: Scaling (Fit on Train)
print("\n--- 2. SCALING ---")
scaler = preprocessing.get_scaler()
X_train_scaled = scaler.fit_transform(X_train)
print("✅ Scaler fitted on 2017.")

In [ ]:
# Cell 4: Train Autoencoder on 2017
print("\n--- 3. TRAIN AUTOENCODER ---")
ae_model = autoencoder.DeepAutoencoder(
    input_dim=config.AE_INPUT_DIM, 
    latent_dim=config.AE_LATENT_DIM,
    hidden_layers=config.AE_HIDDEN_LAYERS
)
# Train AE
# Mới (Lưu vào folder experiment hiện tại)
ae_save_path = exp_paths['models'] / "ae_model_2017.pth"

ae_model = autoencoder.train_ae(ae_model, X_train_scaled, save_path=ae_save_path)

# Extract Latent Features cho Train
print("Extracting latent features for Train set...")
X_train_latent = autoencoder.extract_features(ae_model, X_train_scaled)

In [ ]:
# Cell 5: Prepare Fusion Data for Train
print("\n--- 4. PREPARE FUSION DATA (TRAIN) ---")
# Lấy mRMR features
all_feature_names = config.SELECTED_FEATURES
mrmr_indices = [all_feature_names.index(feat) for feat in config.mRMR_FEATURES]

X_train_mrmr = X_train_scaled[:, mrmr_indices]
X_train_fusion = np.hstack([X_train_mrmr, X_train_latent])

print(f"Fusion Train Shape: {X_train_fusion.shape}")

# Xóa bớt biến nặng để giải phóng RAM cho tập Test
del X_train, X_train_scaled, df_train
gc.collect()

In [ ]:
# Cell 6: Train Random Forest on 2017
print("\n--- 5. TRAIN CLASSIFIER (RF) ---")
rf_save_path = exp_paths['models'] / "rf_model_2017.joblib"
rf_model = rf_classifier.train_rf(X_train_fusion, y_train, save_path=rf_save_path)
print("✅ RF Model Trained on 2017.")

In [ ]:
# ==============================================================================
# CHUYỂN SANG GIAI ĐOẠN TEST TRÊN 2018
# ==============================================================================

# Cell 7: Load Data 2018 (TEST)
print("\n--- 6. LOADING TEST SET (CSE-CIC-IDS2018) ---")
df_test, y_test = preprocessing.load_single_dataset_year('2018', binary_mode=True)
X_test = df_test.values

print(f"Test Data Shape: {X_test.shape}")

In [ ]:
# Cell 8: Process Test Data (Dùng Scaler & AE cũ)
print("\n--- 7. PROCESSING TEST DATA ---")

# A. Scale (Dùng scaler của 2017)
print("Scaling using 2017 scaler...")
X_test_scaled = scaler.transform(X_test)

# B. Extract Latent (Dùng AE của 2017)
print("Extracting latent features...")
X_test_latent = autoencoder.extract_features(ae_model, X_test_scaled)

# C. Slice mRMR Features
print("Slicing mRMR features...")
X_test_mrmr = X_test_scaled[:, mrmr_indices]

# D. Fusion
print("Fusing...")
X_test_fusion = np.hstack([X_test_mrmr, X_test_latent])
print(f"Fusion Test Shape: {X_test_fusion.shape}")

# Clean RAM
del X_test, X_test_scaled, df_test
gc.collect()

In [ ]:
# Cell 9: Evaluate Cross-Dataset
print("\n--- 8. FINAL EVALUATION (Train: 2017 -> Test: 2018) ---")
metrics = evaluation.evaluate_model(
    rf_model, 
    X_test_fusion, 
    y_test, 
    save_dir=exp_paths['figures'],
    dataset_name="Cross-Test (Train 17 - Test 18)"
)

print("\n=== KẾT QUẢ ===")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"MCC:      {metrics['mcc']:.4f}")

In [ ]:
# ==============================================================================
# BATTLE: HYBRID MODEL (AE+RF) vs. PURE RF (BASELINE)
# ==============================================================================

print("\n🥊 BẮT ĐẦU SO SÁNH: PROPOSED METHOD vs. BASELINE 🥊")

# --- 1. Train Baseline (Pure RF với mRMR Features) ---
print("👉 Training Baseline RF (Only mRMR 20 features)...")
# X_train_mrmr đã được tạo ở bước trước (cắt từ X_train_scaled)
# Lưu ý: Cần đảm bảo biến X_train_mrmr còn trong RAM. 
# Nếu lỡ xóa rồi thì phải cắt lại từ X_train_scaled (nếu X_train_scaled cũng xóa thì phải load lại 2017).
# Giả sử bạn chạy liền mạch thì các biến này vẫn còn hoặc có thể tạo lại dễ dàng.

# Cách an toàn nhất để tạo lại dữ liệu train cho Baseline (nếu lỡ xóa RAM):
# (Chỉ chạy đoạn reload này nếu bạn đã lỡ del biến X_train_scaled)
# df_train_temp, y_train_temp = preprocessing.load_single_dataset_year('2017', binary_mode=True)
# scaler_temp = preprocessing.get_scaler()
# X_train_sc_temp = scaler_temp.fit_transform(df_train_temp.values)
# mrmr_indices = [config.SELECTED_FEATURES.index(f) for f in config.MRMR_FEATURES]
# X_train_mrmr = X_train_sc_temp[:, mrmr_indices]
# y_train = y_train_temp
# -------------------------------------------------------------

rf_baseline = rf_classifier.train_rf(
    X_train_mrmr, 
    y_train, 
    save_path=exp_paths['models'] / "rf_baseline_2017.joblib"  # <-- Lưu vào folder exp hiện tại
) 

# --- 2. Evaluate Baseline ---
print("Evaluating Baseline on Test set (2018)...")
# Tương tự, đảm bảo X_test_mrmr có sẵn
baseline_metrics = evaluation.evaluate_model(
    rf_baseline, 
    X_test_mrmr, 
    y_test, 
    save_dir=None,
    dataset_name="Baseline (Pure RF)"
)

# --- 3. So sánh trực tiếp ---
# Lấy kết quả từ metrics cũ (bạn vừa chạy xong)
# Lưu ý: Biến 'metrics' chứa kết quả của Hybrid model vừa chạy ở trên
hybrid_acc = metrics['accuracy'] 
baseline_acc = baseline_metrics['accuracy']

hybrid_mcc = metrics['mcc']
baseline_mcc = baseline_metrics['mcc']

# Lấy thêm Recall Attack để so sánh (Quan trọng nhất)
hybrid_recall = metrics['report']['Attack']['recall']
baseline_recall = baseline_metrics['report']['Attack']['recall']

print("\n📊 === BẢNG KẾT QUẢ ĐỐI ĐẦU (Train 17 -> Test 18) ===")
print(f"{'Metric':<20} | {'Baseline (Pure RF)':<20} | {'Hybrid (Proposed)':<20} | {'Improvement':<15}")
print("-" * 85)
print(f"{'Accuracy':<20} | {baseline_acc:.4f}{'':<14} | {hybrid_acc:.4f}{'':<14} | {hybrid_acc - baseline_acc:+.4f}")
print(f"{'MCC':<20} | {baseline_mcc:.4f}{'':<14} | {hybrid_mcc:.4f}{'':<14} | {hybrid_mcc - baseline_mcc:+.4f}")
print(f"{'Recall (Attack)':<20} | {baseline_recall:.4f}{'':<14} | {hybrid_recall:.4f}{'':<14} | {hybrid_recall - baseline_recall:+.4f}")

# --- 4. Vẽ biểu đồ ---
import matplotlib.pyplot as plt
import numpy as np

metrics_names = ['Accuracy', 'MCC', 'Recall (Attack)']
baseline_scores = [baseline_acc, baseline_mcc, baseline_recall]
hybrid_scores = [hybrid_acc, hybrid_mcc, hybrid_recall]

x = np.arange(len(metrics_names))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width/2, baseline_scores, width, label='Baseline (Pure RF)', color='gray')
rects2 = ax.bar(x + width/2, hybrid_scores, width, label='Hybrid (AE+RF)', color='royalblue')

ax.set_ylabel('Scores')
ax.set_title('Cross-Dataset Comparison: Pure RF vs Hybrid')
ax.set_xticks(x)
ax.set_xticklabels(metrics_names)
ax.legend()
ax.set_ylim([0, 1.05])

def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.4f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.savefig(exp_paths['figures'] / "cross_dataset_comparison.png")
plt.show()